# Principal Component Analysis (PCA) in Python
**Learn about PCA and how it can be leveraged to extract information from the data without any supervision using two popular datasets: Breast Cancer and CIFAR-10.**

## Introduction

Principal Component Analysis (PCA) is a **linear dimensionality reduction** technique that can be utilized for extracting information from a high-dimensional space by projecting it into a lower-dimensional sub-space. It tries to preserve the essential parts that have more variation of the data and remove the non-essential parts with fewer variation.

Dimensions are nothing but features that represent the data. For example, A 28 X 28 image has 784 picture elements (pixels) that are the dimensions or features which together represent that image.

One important thing to note about PCA is that it is an **Unsupervised** dimensionality reduction technique, you can cluster the similar data points based on the feature correlation between them without any supervision (or labels), and you will learn how to achieve this practically using Python in later sections of this tutorial!

According to *Wikipedia*, PCA is a **statistical** procedure that uses an orthogonal transformation to convert a set of observations of possibly correlated variables (entities each of which takes on various numerical values) into a set of values of linearly uncorrelated variables called principal components.

**Note**: Features, Dimensions, and Variables are all referring to the same thing. You will find them being used interchangeably.

#### But where can you apply PCA?

- **Data Visualization**: When working on any data related problem, the challenge in today's world is the sheer volume of data, and the variables/features that define that data. To solve a problem where data is the key, you need extensive data exploration like finding out how the variables are correlated or understanding the distribution of a few variables. Considering that there are a large number of variables or dimensions along which the data is distributed, visualization can be a challenge and almost impossible.

    Hence, PCA can do that for you since it projects the data into a lower dimension, thereby allowing you to visualize the data in a 2D or 3D space with a naked eye.

- **Speeding Machine Learning (ML) Algorithm**: Since PCA's main idea is dimensionality reduction, you can leverage that to speed up your machine learning algorithm's training and testing time considering your data has a lot of features, and the ML algorithm's learning is too slow.

At an abstract level,  you take a dataset having many features, and you simplify that dataset by selecting a few ``Principal Components`` from original features.

#### What is a Principal Component?

Principal components are the key to PCA; they represent what's underneath the hood of your data. In a layman term, when the data is projected into a lower dimension (assume three dimensions) from a higher space, the three dimensions are nothing but the three Principal Components that captures (or holds) most of the variance (information) of your data.

Principal components have both direction and magnitude. The direction represents across which *principal axes* the data is mostly spread out or has most variance and the magnitude signifies the amount of variance that Principal Component captures of the data when projected onto that axis. The principal components are a straight line, and the first principal component holds the most variance in the data. Each subsequent principal component is orthogonal to the last and has a lesser variance. In this way, given a set of <i>x</i> correlated variables over <i>y</i> samples you achieve a set of <i>u</i> uncorrelated principal components over the same <i>y</i> samples.

The reason you achieve uncorrelated principal components from the original features is that the correlated features contribute to the same principal component, thereby reducing the original data features into uncorrelated principal components; each representing a different set of correlated features with different amounts of variation.

Each principal component represents a percentage of total variation captured from the data.

In today's tutorial, you will mainly apply PCA on the two use-cases:
- ``Data Visualization``
- ``Speeding ML algorithm``

To accomplish the above two tasks, you will use two famous Breast Cancer (numerical) and CIFAR - 10 (image) dataset.

## Understanding the Data

Before you go ahead and load the data, it's good to understand and look at the data that you will be working with!

### Breast Cancer

The Breast Cancer data set is a real-valued multivariate data that consists of two classes, where each class signifies whether a patient has breast cancer or not. The two categories are: malignant and benign.

The malignant class has 212 samples, whereas the benign class has 357 samples.

It has 30 features shared across all classes: radius, texture, perimeter, area, smoothness, fractal dimension, etc.

You can download the breast cancer dataset from <a href="https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)">here</a>, or rather an easy way is by loading it with the help of the ``sklearn`` library.

### CIFAR - 10

The CIFAR-10 (Canadian Institute For Advanced Research) dataset consists of 60000 images each of 32x32x3 color images having ten classes, with 6000 images per category.

The dataset consists of 50000 training images and 10000 test images.

The classes in the dataset are airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck.

You can download the CIFAR dataset from <a href="https://www.cs.toronto.edu/~kriz/cifar.html">here</a>, or you can also load it on the fly with the help of a deep learning library like ``Keras``.

## Data Exploration

Now you will be loading and analyzing the ``Breast Cancer`` and ``CIFAR-10`` datasets. By now you have an idea regarding the dimensionality of both datasets.

So, let's quickly explore both datasets.

#### Breast Cancer Data Exploration

Let's first explore the ``Breast Cancer`` dataset.

You will use ``sklearn's`` module ``datasets`` and import the ``Breast Cancer`` dataset from it.


In [ ]:
from sklearn.datasets import load_breast_cancer

``load_breast_cancer`` will give you both labels and the data. To fetch the data, you will call ``.data`` and for fetching the labels ``.target``.

The data has 569 samples with thirty features, and each sample has a label associated with it. There are two labels in this dataset.


In [ ]:
breast = load_breast_cancer()

In [ ]:
breast_data = breast.data

Let's check the shape of the data.


In [ ]:
breast_data.shape

Even though for this tutorial, you do not need the labels but still for better understanding, let's load the labels and check the shape.


In [ ]:
breast_labels = breast.target

In [ ]:
breast_labels.shape

Now you will import ``numpy`` since you will be reshaping the ``breast_labels`` to concatenate it with the ``breast_data`` so that you can finally create a ``DataFrame`` which will have both the data and labels.


In [ ]:
import numpy as np

In [ ]:
labels = np.reshape(breast_labels,(569,1))

After ``reshaping`` the labels, you will ``concatenate`` the data and labels along the second axis, which means the final shape of the array will be ``569 x 31``.


In [ ]:
final_breast_data = np.concatenate([breast_data,labels],axis=1)

In [ ]:
final_breast_data.shape

Now you will import ``pandas`` to create the ``DataFrame`` of the final data to represent the data in a tabular fashion.


In [ ]:
import pandas as pd

In [ ]:
breast_dataset = pd.DataFrame(final_breast_data)

Let's quickly print the features that are there in the breast cancer dataset!


In [ ]:
features = breast.feature_names

In [ ]:
features

If you note in the ``features`` array, the ``label`` field is missing. Hence, you will have to manually add it to the ``features`` array since you will be equating this array with the column names of your ``breast_dataset`` dataframe.


In [ ]:
features_labels = np.append(features,'label')

Great! Now you will embed the column names to the ``breast_dataset`` dataframe.


In [ ]:
breast_dataset.columns = features_labels

Let's print the first few rows of the dataframe.


In [ ]:
breast_dataset.head()

Since the original labels are in ``0,1`` format, you will change the labels to ``benign`` and ``malignant`` using ``.replace`` function. You will use ``inplace=True`` which will modify the dataframe ``breast_dataset``.


In [ ]:
breast_dataset['label'].replace(0, 'Benign',inplace=True)
breast_dataset['label'].replace(1, 'Malignant',inplace=True)

Let's print the last few rows of the ``breast_dataset``.


In [ ]:
breast_dataset.tail()

#### CIFAR - 10  Data Exploration

Next, you'll explore the ``CIFAR - 10`` image dataset

You can load the ``CIFAR - 10`` dataset using a deep learning library called ``Keras``.


In [ ]:
from keras.datasets import cifar10

Once imported, you will use the ``.load_data()`` method to download the data, it will download and store the data in your ``Keras`` directory. This can take some time based on your internet speed.


In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

The above line of code returns training and test images along with the labels.

Let's quickly print the shape of training and testing images shape.

**Note:** Before we do that we make sure to only use a subset of the data. The results won't be all too different, but it will speed up our runtime and it as such will suffice for educational purposes.


In [ ]:
(x_train, y_train), (x_test, y_test) = (x_train[:800], y_train[:800]), (x_test[:200], y_test[:200])

In [ ]:
print('Traning data shape:', x_train.shape)
print('Testing data shape:', x_test.shape)

Let's also print the shape of the labels.


In [ ]:
y_train.shape,y_test.shape

Let's also find out the total number of labels and the various kinds of classes the data has.


In [ ]:
# Find the unique numbers from the train labels
classes = np.unique(y_train)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Now to plot the ``CIFAR-10`` images, you will import ``matplotlib`` and also use a ``magic (%)`` command ``%matplotlib inline`` to tell the jupyter notebook to show the output within the notebook itself!


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

For a better understanding, let's create a dictionary that will have class names with their corresponding categorical class labels.


In [ ]:
label_dict = {
 0: 'airplane',
 1: 'automobile',
 2: 'bird',
 3: 'cat',
 4: 'deer',
 5: 'dog',
 6: 'frog',
 7: 'horse',
 8: 'ship',
 9: 'truck',
}

In [ ]:
plt.figure(figsize=[5,5])

# Display the first image in training data
plt.subplot(121)
curr_img = np.reshape(x_train[0], (32,32,3))
plt.imshow(curr_img)
print(plt.title("(Label: " + str(label_dict[y_train[0][0]]) + ")"))

# Display the first image in testing data
plt.subplot(122)
curr_img = np.reshape(x_test[0],(32,32,3))
plt.imshow(curr_img)
print(plt.title("(Label: " + str(label_dict[y_test[0][0]]) + ")"))

Even though the above two images are blurry, you can still somehow observe that the first image is a frog with the label ``frog``, while the second image is of a cat with the label ``cat``.

## Data Visualization using PCA

Now comes the most exciting part of this tutorial. As you learned earlier that PCA projects turn high-dimensional data into a low-dimensional principal component, now is the time to visualize that with the help of Python!

### Visualizing the Breast Cancer data

- You start by <b>``Standardizing``</b> the data since PCA's output is influenced based on the scale of the features of the data.
- It is a common practice to normalize your data before feeding it to any machine learning algorithm.

- To apply normalization, you will import ``StandardScaler`` module from the sklearn library and select only the features from the ``breast_dataset`` you created in the Data Exploration step. Once you have the features, you will then apply scaling by doing ``fit_transform`` on the feature data.

- While applying StandardScaler, each feature of your data should be normally distributed such that it will scale the distribution to a mean of zero and a standard deviation of one.



In [ ]:
from sklearn.preprocessing import StandardScaler
x = breast_dataset.loc[:, features].values
x = StandardScaler().fit_transform(x) # normalizing the features

In [ ]:
x.shape

Let's check whether the normalized data has a mean of zero and a standard deviation of one.


In [ ]:
np.mean(x),np.std(x)

Let's convert the normalized features into a tabular format with the help of DataFrame.


In [ ]:
feat_cols = ['feature' + str(i) for i in range(x.shape[1])]

In [ ]:
normalised_breast = pd.DataFrame(x,columns=feat_cols)

In [ ]:
normalised_breast.tail()

- Now comes the critical part, the next few lines of code will be projecting the thirty-dimensional Breast Cancer data to two-dimensional <b>``principal components``</b>.

- You will use the sklearn library to import the ``PCA`` module, and in the PCA method, you will pass the number of components (n_components=2) and finally call fit_transform on the aggregate data. Here, several components represent the lower dimension in which you will project your higher dimension data.



In [ ]:
from sklearn.decomposition import PCA
pca_breast = PCA(n_components=2)
principalComponents_breast = pca_breast.fit_transform(x)

Next, let's create a DataFrame that will have the principal component values for all 569 samples.


In [ ]:
principal_breast_Df = pd.DataFrame(data = principalComponents_breast
             , columns = ['principal component 1', 'principal component 2'])

In [ ]:
principal_breast_Df.tail()

- Once you have the principal components, you can find the <b>``explained_variance_ratio``</b>. It will provide you with the amount of information or variance each principal component holds after projecting the data to a lower dimensional subspace.


In [ ]:
print('Explained variation per principal component: {}'.format(pca_breast.explained_variance_ratio_))

From the above output, you can observe that the ``principal component 1`` holds 44.2% of the information while the ``principal component 2`` holds only 19% of the information. Also, the other point to note is that while projecting thirty-dimensional data to a two-dimensional data, 36.8% information was lost.

Let's plot the visualization of the 569 samples along the ``principal component - 1`` and ``principal component - 2`` axis. It should give you good insight into how your samples are distributed among the two classes.


In [ ]:
plt.figure()
plt.figure(figsize=(10,10))
plt.xticks(fontsize=12)
plt.yticks(fontsize=14)
plt.xlabel('Principal Component - 1',fontsize=20)
plt.ylabel('Principal Component - 2',fontsize=20)
plt.title("Principal Component Analysis of Breast Cancer Dataset",fontsize=20)
targets = ['Benign', 'Malignant']
colors = ['r', 'g']

for target, color in zip(targets,colors):
    indicesToKeep = breast_dataset['label'] == target
    plt.scatter(principal_breast_Df.loc[indicesToKeep, 'principal component 1']
               , principal_breast_Df.loc[indicesToKeep, 'principal component 2'], c = color, s = 50)

plt.legend(targets,prop={'size': 15});

From the above graph, you can observe that the two classes ``benign`` and ``malignant``, when projected to a two-dimensional space, can be linearly separable up to some extent. Other observations can be that the ``benign`` class is spread out as compared to the ``malignant`` class.

### Visualizing the CIFAR - 10 data

The following lines of code for visualizing the CIFAR-10 data is pretty similar to the PCA visualization of the Breast Cancer data.

- Let's quickly check the maximum and minimum values of the CIFAR-10 training images and <b>``normalize``</b> the pixels between 0 and 1 inclusive.


In [ ]:
np.min(x_train),np.max(x_train)

In [ ]:
x_train = x_train/255.0

In [ ]:
np.min(x_train),np.max(x_train)

In [ ]:
x_train.shape

Next, you will create a DataFrame that will hold the pixel values of the images along with their respective labels in a row-column format.

But before that, let's reshape the image dimensions from three to one (flatten the images).


In [ ]:
x_train_flat = x_train.reshape(-1,3072)

In [ ]:
feat_cols = ['pixel'+str(i) for i in range(x_train_flat.shape[1])]

In [ ]:
df_cifar = pd.DataFrame(x_train_flat,columns=feat_cols)

In [ ]:
df_cifar['label'] = y_train
print('Size of the dataframe: {}'.format(df_cifar.shape))

Perfect! The size of the dataframe is correct since there are 50,000 training images, each having 3072 pixels and an additional column for labels so in total 3073.

PCA will be applied on all the columns except the last one, which is the label for each image.


In [ ]:
df_cifar.head()

- Next, you will create the PCA method and pass the number of components as two and apply ``fit_transform`` on the training data, this can take few seconds since there are 50,000 samples


In [ ]:
pca_cifar = PCA(n_components=2)
principalComponents_cifar = pca_cifar.fit_transform(df_cifar.iloc[:1000,:-1])

Then you will convert the principal components for each of the 50,000 images from a numpy array to a pandas DataFrame.


In [ ]:
principal_cifar_Df = pd.DataFrame(data = principalComponents_cifar
             , columns = ['principal component 1', 'principal component 2'])
principal_cifar_Df['y'] = y_train

In [ ]:
principal_cifar_Df.head()

- Let's quickly find out the amount of information or ``variance`` the principal components hold.


In [ ]:
print('Explained variation per principal component: {}'.format(pca_cifar.explained_variance_ratio_))

    Explained variation per principal component: [0.2907663  0.11253144]

Well, it looks like a decent amount of information was retained by the principal components 1 and 2, given that the data was projected from 3072 dimensions to a mere two principal components.

Its time to visualize the CIFAR-10 data in a two-dimensional space. Remember that there is some semantic class overlap in this dataset which means that a frog can have a slightly similar shape of a cat or a deer with a dog; especially when projected in a two-dimensional space. The differences between them might not be captured that well.


In [ ]:
import seaborn as sns
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="principal component 1", y="principal component 2",
    hue="y",
    palette=sns.color_palette("hls", 10),
    data=principal_cifar_Df,
    legend="full",
    alpha=0.3
)

From the above figure, you can observe that some variation was captured by the principal components since there is some structure in the points when projected along the two principal component axis. The points belonging to the same class are close to each other, and the points or images that are very different semantically are further away from each other.

## Speed Up Deep Learning Training using PCA with CIFAR - 10 Dataset

In this final segment of the tutorial, you will be learning about how you can speed up your Deep Learning Model's training process using PCA.

**Note**: To learn basic terminologies that will be used in this section, please feel free to check out <a href="
https://www.datacamp.com/community/tutorials/convolutional-neural-networks-python">this tutorial</a>.

First, let's normalize the training and testing images. If you remember the training images were normalized in the PCA visualization part, so you only need to normalize the testing images. So, let's quickly do that!


In [ ]:
x_test = x_test/255.0

In [ ]:
x_test = x_test.reshape(-1,32,32,3)

Let's ``reshape`` the test data.


In [ ]:
x_test_flat = x_test.reshape(-1,3072)

Next, you will make the instance of the PCA model.

Here, you can also pass how much variance you want PCA to capture. Let's pass 0.9 as a parameter to the PCA model, which means that PCA will hold 90% of the variance and the ``number of components`` required to capture 90% variance will be used.

Note that earlier you passed ``n_components`` as a parameter and you could then find out how much variance was captured by those two components. But here we explicitly mention how much variance we would like PCA to capture and hence, the ``n_components`` will vary based on the variance parameter.

If you do not pass any variance, then the number of components will be equal to the original dimension of the data.


In [ ]:
pca = PCA(0.9)

Then you will fit the ``PCA`` instance on the training images.


In [ ]:
pca.fit(x_train_flat)

Now let's find out how many ``n_components`` PCA used to capture 0.9 variance.


In [ ]:
pca.n_components_

From the above output, you can observe that to achieve 90% variance, the dimension was reduced to ``83`` principal components from the actual ``3072`` dimensions.

Finally, you will apply ``transform`` on both the training and test set to generate a transformed dataset from the parameters generated from the ``fit`` method.


In [ ]:
train_img_pca = pca.transform(x_train_flat)
test_img_pca = pca.transform(x_test_flat)

Next, let's quickly import the necessary libraries to run the deep learning model.


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from tensorflow.keras.optimizers import RMSprop

Now, you will convert your training and testing labels to one-hot encoding vector.


In [ ]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

Let's define the number of epochs, number of classes, and the batch size for your model.


In [ ]:
batch_size = 128
num_classes = 10
epochs = 20

Next, you will define your ``Sequential`` model!


In [ ]:
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(83,)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

Let's print the model summary.


In [ ]:
model.summary()

Finally, it's time to compile and train the model!


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(train_img_pca, y_train,batch_size=batch_size,epochs=epochs,verbose=1,
                    validation_data=(test_img_pca, y_test))

From the above output, you can observe that the time taken for training each epoch was just ``7 seconds`` on a CPU. The model did a decent job on the training data, achieving ``70%`` accuracy while it achieved only ``56%`` accuracy on the test dat. This means that it overfitted the training data. However, remember that the data was projected to 99 dimensions from 3072 dimensions and despite that it did a great job!

Finally, let's see how much time the model takes to train on the original dataset and how much accuracy it can achieve using the same deep learning model.


In [ ]:
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(3072,)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train_flat, y_train,batch_size=batch_size,epochs=epochs,verbose=1,
                    validation_data=(x_test_flat, y_test))

Voila! From the above output, it is quite evident that the time taken for training each epoch was around ``23 seconds`` on a CPU which was almost three times more than the model trained on the PCA output.

Moreover, both the training and testing accuracy is less than the accuracy you achieved with the 99 principal components as an input to the model.

So, by applying PCA on the training data you were able to train your deep learning algorithm not only ``fast``, but it also achieved better ``accuracy`` on the testing data when compared with the deep learning algorithm trained with original training data.

## Go Further!

Congratulations on finishing the tutorial.

This tutorial was an excellent and comprehensive introduction to PCA in Python, which covered both the theoretical, as well as, the practical concepts of PCA.

If you want to dive deeper into dimensionality reduction techniques then consider reading about t-distributed Stochastic Neighbor Embedding commonly known as <a href="
https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html">tSNE</a>, which is a non-linear probabilistic dimensionality reduction technique.

If you would like to learn more about unsupervised learning techniques like PCA, take DataCamp's <a href="https://www.datacamp.com/courses/unsupervised-learning-in-python">Unsupervised Learning in Python</a> course.

References:

 - <a href="https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html">PCA in Sklearn</a>
 - <a href="https://www.datacamp.com/community/tutorials/pca-analysis-r">Principal Component Analysis in R</a>
 - <a href="https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60">PCA using Python (scikit-learn)</a>

Please feel free to ask any questions related to this tutorial in the comments section below.